In [4]:
import sqlite3
import pandas as pd

### sqlite3 db to Dataframe

In [5]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()

In [6]:
df_store = pd.read_sql_query("SELECT * from api_store", con)
df_review = pd.read_sql_query("SELECT * from api_review", con)

In [7]:
df_store.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,0,0
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,0,0
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,0,0
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,0,0
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,0,0


In [8]:
df_review.head()

,id,total_score,content,reg_time,store_id,user_id
0,1,5.0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,2020-04-06 01:19:01.604070,15,68632
1,2,5.0,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,2020-04-06 01:19:01.605069,18,389728
2,3,4.0,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,2020-04-06 01:19:01.605069,19,68716
3,4,2.0,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",2020-04-06 01:19:01.605069,37,774353
4,5,3.0,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2020-04-06 01:19:01.605069,38,115682


In [9]:
con.close()

### surprise
#### https://medium.com/sfu-cspmp/recommendation-systems-user-based-collaborative-filtering-using-n-nearest-neighbors-bf7361dc24e0
#### https://github.com/ashaypathak/Recommendation-system/blob/master/Movie_Recommendation.ipynb

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [11]:
stores = df_store[['id','store_name','category','address']]
reviews = df_review[['user_id','store_id','total_score']]
reviews.rename(columns={'total_score':'rating'}, inplace=True)
stores.rename(columns={'id':'store_id'},inplace=True)

C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\pandas\core\frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [ ]:
Mean = reviews.groupby(by="user_id",as_index=False)['rating'].mean()
Rating_avg = pd.merge(reviews,Mean,on='user_id')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

In [ ]:
check = pd.pivot_table(Rating_avg,values='rating_x',index='user_id',columns='store_id')
check.head()

In [ ]:
final = pd.pivot_table(Rating_avg,values='adg_rating',index='user_id',columns='store_id')
final.head()

In [ ]:
# Replacing NaN by Movie Average
final_store = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [ ]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

In [ ]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_store)
np.fill_diagonal(cosine, 0 )
similarity_with_store = pd.DataFrame(cosine,index=final_store.index)
similarity_with_store.columns=final_user.index
similarity_with_store.head()

In [ ]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
# top 30 neighbours for each user
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head()

In [ ]:
# top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_store,30)
sim_user_30_m.head()

In [ ]:
Rating_avg = Rating_avg.astype({"store_id": str})
Movie_user = Rating_avg.groupby(by = 'user_id')['store_id'].apply(lambda x:','.join(x))

In [ ]:
def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_store.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['user_id'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_store.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'store_id':Movies_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    Movie_Name = top_5_recommendation.merge(stores, how='inner', on='store_id')
    print(Movie_Name)
    Movie_Names = Movie_Name.store_name.values.tolist()
    return Movie_Names

In [ ]:
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for User Id :"+str(user))
print("   ")
for i in predicted_movies:
    print(i)

In [ ]:
df[df['uid']==4974]

In [ ]:
stores[stores['store_name']=='뚝닭']


*** 절취선 ***

In [ ]:
df[df['iid']==116469]

#### cross_validate parameters 
+ algo ( AlgoBase) – 평가할 알고리즘입니다.
+ data ( Dataset) – 알고리즘을 평가할 데이터 세트입니다.
측정 값 ( 문자열 목록 ) – 계산할 성능 측정 값입니다. 허용되는 이름은 accuracy모듈에 정의 된 기능 이름 입니다. 기본값은 입니다.['rmse', 'mae']
+ cv (cross-validation iterator, int 또는 None) – data매개 변수가 분할되는 방법 (트레인 셋 및 테스트 세트가 정의되는 방법)을 결정합니다. int가 전달되면 KFold적절한 n_splits매개 변수 와 함께 사용됩니다 . 경우 None, KFold사용됩니다 n_splits=5.
+ return_train_measures ( bool ) – 기차 세트에서 성능 측정 값을 계산할지 여부입니다. 기본값은 False입니다.
n_jobs ( int ) –
병렬로 평가 된 최대 접기 수입니다.

인 경우 -1모든 CPU가 사용됩니다.
1주어진 경우 병렬 컴퓨팅 코드가 전혀 사용되지 않으므로 디버깅에 유용합니다.
내용은 n_jobs아래에 -1, 사용된다. 예를 들어 하나의 CPU를 제외한 모든 CPU가 사용됩니다.(n_cpus + n_jobs + 1)n_jobs = -2
기본값은 1입니다.

+ pre_dispatch ( int 또는 string ) –
병렬 실행 중에 디스패치되는 작업 수를 제어합니다. 이 수를 줄이면 CPU가 처리 할 수있는 것보다 더 많은 작업이 디스패치 될 때 메모리 소비가 급격히 증가하는 것을 피할 수 있습니다. 이 매개 변수는 다음과 같습니다.

None이 경우 모든 작업이 즉시 생성되어 생성됩니다. 작업의 주문형 생성으로 인한 지연을 피하기 위해 가볍고 빠르게 실행되는 작업에 사용하십시오.
생성 된 정확한 총 작업 수를 제공하는 정수입니다.
에서와 같이의 함수로 표현식을 제공하는 문자열 n_jobs입니다 '2*n_jobs'.
기본값은 '2*n_jobs'입니다.

+ verbose ( int ) – True각 분할에 대한 정확도 측정 값이 인쇄되고 훈련 및 테스트 시간 모든 분할에 대한 평균 및 표준 편차도보고됩니다. 기본값은 False아무것도 인쇄되지 않습니다.

https://www.fun-coding.org/recommend_basic7.html

In [12]:
df =pd.merge(stores, reviews,on="store_id")
df.head()

,store_id,store_name,category,address,user_id,rating
0,15,써리힐,호주레스토랑,부산광역시 부산진구 전포동 229-13번지 2층,68632,5.0
1,18,진삼미 샌드위치,샌드위치|쥬스,전라북도 전주시 완산구 효자동1가 652 상산고등학교앞,389728,5.0
2,19,한옥마을 전주비빔밥,전주비빔밥,경기도 화성시 향남읍 행정리 487-1 홈플러스 화성향남점 1층,68716,4.0
3,37,007식당,굴비정식|굴비,전라남도 영광군 법성면 법성리 1148-6,774353,2.0
4,38,010수산,횟집,서울특별시 강서구 등촌동 671-3,115682,3.0


In [13]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import dump
import surprise
import os

#### Memory error ... build_anti_testset()에서 나는듯..

In [14]:
#swapping columns
df = pd.merge(reviews, stores,on="store_id")
raw=df[['user_id','store_id','rating']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(0,5)) 
#into surprise:
dataframe = surprise.Dataset.load_from_df(raw,reader)
trainset = dataframe.build_full_trainset()
# Use the famous SVD algorithm.
algo = SVD()
algo.fit(trainset)

In [13]:
testset = trainset.build_anti_testset()

MemoryError: 

In [12]:
predictions = algo.test(trainset.build_anti_testset())

MemoryError: 

In [ ]:
# Dump algorithm and reload it.
file_name = os.path.expanduser('dump_file')
dump.dump(file_name, algo=algo)
_, loaded_algo = dump.load(file_name)

# We now ensure that the algo is still the same by checking the predictions.
predictions_loaded_algo = loaded_algo.test(trainset.build_testset())
assert predictions == predictions_loaded_algo
print('Predictions are the same')

In [ ]:
from collections import defaultdict
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
reviews[reviews['user_id']==183349]

In [ ]:
reviews[reviews['store_id']==24772]

In [ ]:
algo = surprise.SVD()
cross_validate(algo, dataframe,measures=['RMSE','MAE'], cv=5, verbose=True)

#### https://lsjsj92.tistory.com/570?category=853217

#### https://antilibrary.org/2086

In [ ]:
from surprise import SVD,SVDpp, SlopeOne, NMF,NormalPredictor,KNNBaseline,KNNBasic,KNNWithMeans,KNNWithZScore,BaselineOnly,CoClustering
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import dump
#swapping columns
raw=df[['user_id','store_id','rating']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(0,5)) 
#into surprise:
dataframe = Dataset.load_from_df(raw,reader)

In [ ]:
algo = surprise.SVD()
# cross_validate(algo, dataframe,measures=['RMSE'], cv=3, verbose=True)
algo.fit(dataframe.build_full_trainset())

In [ ]:
output

In [ ]:
pred = algo.predict(uid='17371',iid="36441",r_ui=4, verbose=True)
score = pred.est
print(score)

In [ ]:
df.head()

In [15]:
test = df[df['address'].str.contains("서울")]

In [16]:
test

,user_id,store_id,rating,store_name,category,address
4,115682,38,3.0,010수산,횟집,서울특별시 강서구 등촌동 671-3
5,388883,49,5.0,03수제비칼국수,수제비,서울특별시 서초구 방배동 768-2
6,180541,49,4.0,03수제비칼국수,수제비,서울특별시 서초구 방배동 768-2
7,249051,50,5.0,03인앤아웃,샌드위치|터키샌드위치,서울특별시 서대문구 연희동 126-2
26,872014,92,4.0,10 꼬르소꼬모 카페,cafe|브런치,서울특별시 강남구 청담동 79 Trinity 빌딩 지하 1층
...,...,...,...,...,...,...
91388,574786,360492,4.0,지구촌,루프탑|라자냐,서울특별시 용산구 이태원동 130-43
91389,419534,360493,5.0,지구촌 가정식,미고랭|밥집,서울특별시 마포구 연남동 228-4
91390,43793,360493,4.0,지구촌 가정식,미고랭|밥집,서울특별시 마포구 연남동 228-4
91391,5239,360493,3.0,지구촌 가정식,미고랭|밥집,서울특별시 마포구 연남동 228-4


In [17]:
iids = test['store_id'].unique()
iids50 = raw.loc[raw['user_id']==4974, 'store_id']
iids_to_pred = np.setdiff1d(iids,iids50) # 안 간 가게 구함(차집합)

In [19]:
testset = [[4974,iid,5.] for iid in iids_to_pred]
predictions = algo.test(testset)
predictions[0]

Prediction(uid=4974, iid=38, r_ui=5.0, est=3.551184984354573, details={'was_impossible': False})

In [20]:
pred_ratings = np.array([pred.est for pred in predictions])
i_max = pred_ratings.argsort()[:10]
iid = iids_to_pred[i_max]
print('Top store for user has store_id {0} with predicted rating {1}'.format(iid,pred_ratings[i_max]))

Top store for user has store_id [281035  25753 242048 264422 119891  35887 330067 242043 139063 194980] with predicted rating [2.48736304 2.49441277 2.59356822 2.63789005 2.6716175  2.67987854
 2.68567142 2.71473322 2.71551675 2.75742525]


In [21]:
for i,m in zip(iid,i_max):
    print()
    print('{0} : {1}'.format(i,pred_ratings[m]))


281035 : 2.4873630377770093

25753 : 2.4944127719269935

242048 : 2.593568216219813

264422 : 2.63789005394074

119891 : 2.671617502703624

35887 : 2.6798785441143482

330067 : 2.68567141791126

242043 : 2.7147332180770785

139063 : 2.71551674687952

194980 : 2.757425249129829


In [ ]:
df_store[df_store['id'].isin(iid)]

In [ ]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0

In [ ]:
# 정확도 계산
from surprise import accuracy

testset = trainset.build_testset()
predictions = algo.test(testset)

accuracy.rmse(predictions)

In [ ]:
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [ ]:
s = raw[raw['user_id']==4974]

In [ ]:
s=pd.DataFrame(s)

In [ ]:
s[s['store_id'].isin(iid)]

In [ ]:
df